## Reconcile Products Table
#### Contributors:
##### Tean 8: Anthony Ung, Sean Jerzewski, Gideon Kipkorir

## 0. Dependencies and Global Variables

In [1]:
import os
from enum import Enum
import csv
import sqlite3 as lite
from decimal import Decimal
from datetime import date, datetime, timedelta

In [2]:
DB_HANDLES = {}

## 1. Gather the file paths
  
  
## IMPORTANT: 
#### Most of these files are untracked on GitHub. it is each team members'   
####   &emsp; &emsp; It is each team members' individual responsibilities  
####   &emsp; &emsp; to build the Database and CSV files for themselves using the other Jupyter notebooks. 

In [3]:
FILE_PATHS = {
    'DATA_MART' : './../01_Source_DBs/Region_C_Data_Mart.db'
}

DB_HANDLES = {}

In [4]:
ALL_FILES_OK = True

for file_key in FILE_PATHS:
    file_name = FILE_PATHS[file_key]
    file_exists = os.path.isfile(file_name)
    
    if(file_exists):
        print(f'OK - {file_key} - \'{file_name}\'')
    else:
        ALL_FILES_OK = False
        print(f'MISSING - {file_key} - \'{file_name}\'')

if not ALL_FILES_OK:
    raise SystemExit('\n' "ERROR!" '\n' "You are missing files!" '\n' "Read and Follow the Cell instructions provided.")

OK - DATA_MART - './../01_Source_DBs/Region_C_Data_Mart.db'


## 2. Initialize the Database File and the Database API

I originally made this Database API back in HW 2 and developed it throughout the semester

In [5]:
'''
    If I try to make db_options an inner class to db, 
        I get an error saying that the class is undefined.
'''
class db_options(Enum):
        DEFAULT = 0
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2

class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql, values)


    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

In [6]:
DB_HANDLES['DATA_MART'] = db(FILE_PATHS['DATA_MART'])

## 3. Top Selling Items for 2024 Q1 and 2025 Q1

In [7]:
def run():
    db_handle = DB_HANDLES['DATA_MART']

    sql = \
        '''
        WITH combined_sales AS (
            SELECT
                RANK() OVER (ORDER BY iq.TotalSoldByStoreThisQuarter DESC) AS CombinedRank,
                SUM(iq.TotalSoldByStoreThisQuarter) AS CombinedSold,
                p.sku,
                p.product_name,
                p.brandName,
                p.subcategory
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1)
            GROUP BY iq.ProductKey
            ORDER BY CombinedRank ASC
        ),
        sales_2024 AS (
            SELECT
                p.sku,
                p.product_name,
                p.brandName,
                p.subcategory,
                RANK() OVER (ORDER BY iq.TotalSoldByStoreThisQuarter DESC) AS Rank2024Q1,
                SUM(iq.TotalSoldByStoreThisQuarter) AS Sales2024Q1
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1) AND (Year = 2024)
            GROUP BY iq.ProductKey
            ORDER BY Rank2024Q1 ASC
        ), 
        sales_2025 AS (
            SELECT
                p.sku,
                p.product_name,
                p.brandName,
                p.subcategory,
                RANK() OVER (ORDER BY iq.TotalSoldByStoreThisQuarter DESC) AS Rank2025Q1,
                SUM(iq.TotalSoldByStoreThisQuarter) AS Sales2025Q1
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1) AND (Year = 2025)
            GROUP BY iq.ProductKey
            ORDER BY Rank2025Q1 ASC
        )
        
        SELECT cs.*,
            s_2024.Rank2024Q1,
            s_2024.Sales2024Q1,
            s_2025.Rank2025Q1,
            s_2025.Sales2025Q1
        FROM combined_sales AS cs
        JOIN sales_2024 AS s_2024 ON (cs.sku = s_2024.sku)
        JOIN sales_2025 AS s_2025 ON (cs.sku = s_2025.sku)
        WHERE (cs.CombinedRank <= 25) OR (s_2024.Rank2024Q1 <= 25) OR (s_2025.Rank2025Q1 <= 25);
        '''

    db_handle.connect()
    TOP_PRODUCTS = db_handle.execute_sql(sql, options=db_options.RETURN_RESULTS)
    for row in TOP_PRODUCTS:
        print(row)

    with open('Team_8_Top_25_Products.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
    
        headers = ['Combined_Rank', 'Combined_Sold', 'SKU', 'Product_Name', 'Product_Brand', 'Product_Subcategory', \
                   '2024_Rank', '2024_Sold', '2025_Rank', '2025_Sold']
    
        writer.writerow(headers)
        writer.writerows(TOP_PRODUCTS)
    db_handle.close()
    
run()

(1, 45678.0, '42357001', '2.00% Milk', 'Rowan Dairy', 'Milk', 1, 34573.0, 5, 11105.0)
(2, 45602.0, '42358001', '2.00% Milk', 'Rowan Dairy', 'Milk', 2, 34564.0, 6, 11038.0)
(3, 45524.0, '42355001', '1.00% Milk', 'Rowan Dairy', 'Milk', 3, 34289.0, 2, 11235.0)
(4, 45421.0, '42360001', 'Whole Milk Milk', 'Rowan Dairy', 'Milk', 4, 34175.0, 1, 11246.0)
(5, 45635.0, '42356001', '1.00% Milk', 'Rowan Dairy', 'Milk', 5, 34466.0, 4, 11169.0)
(6, 45299.0, '42359001', 'Whole Milk Milk', 'Rowan Dairy', 'Milk', 6, 34070.0, 3, 11229.0)
(7, 12883.0, '42311001', 'Jam Grape', 'Smuckers', 'Jam', 7, 9778.0, 8, 3105.0)
(8, 12771.0, '42313001', 'Jelly Grape', 'Smuckers', 'Jam', 8, 9635.0, 7, 3136.0)
(9, 12949.0, '42312001', 'Jam Strawberry', 'Smuckers', 'Jam', 9, 9853.0, 9, 3096.0)
(10, 12805.0, '42314001', 'Squeeze Jelly Grape', 'Smuckers', 'Jelly', 10, 9709.0, 9, 3096.0)
(11, 9030.0, '43895001', 'Booker Blueberry Yogurt', 'Booker', 'Yogurt', 11, 6522.0, 638, 2508.0)
(12, 8981.0, '42240001', 'Pie Peach', 'T

## 4. Top Selling Categories for 2024 Q1 and 2025 Q1

In [8]:
def run():
    db_handle = DB_HANDLES['DATA_MART']

    sql = \
        '''
        WITH combined_sales AS (
            WITH cs_1 AS (
            SELECT
                SUM(iq.TotalSoldByStoreThisQuarter) AS CombinedSold,
                p.subcategory,
                p.category
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1)
            GROUP BY p.subcategory
            ORDER BY CombinedSold DESC
            )
            SELECT RANK() OVER (ORDER BY CombinedSold DESC) AS rank,
                *
            FROM cs_1
        ), 
        sales_2024 AS (
            WITH cs_1 AS (
            SELECT
                SUM(iq.TotalSoldByStoreThisQuarter) AS CombinedSold,
                p.subcategory,
                p.category
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1) AND (Year = 2024)
            GROUP BY p.subcategory
            ORDER BY CombinedSold DESC
            )
            SELECT RANK() OVER (ORDER BY CombinedSold DESC) AS rank,
                *
            FROM cs_1
        ),
        sales_2025 AS (
            WITH cs_1 AS (
            SELECT
                SUM(iq.TotalSoldByStoreThisQuarter) AS CombinedSold,
                p.subcategory,
                p.category
            FROM inventory_quarterly AS iq
            JOIN products AS p ON (iq.ProductKey = p.ProductKey1)
            WHERE (iq.Quarter = 1) AND (Year = 2025)
            GROUP BY p.subcategory
            ORDER BY CombinedSold DESC
            )
            SELECT RANK() OVER (ORDER BY CombinedSold DESC) AS rank,
                *
            FROM cs_1
        )
        SELECT 
            cs.*,
            s_2024.rank AS rank_2024,
            s_2024.CombinedSold AS sold_2024,
            s_2025.rank AS rank_2025,
            s_2025.CombinedSold AS sold_2025
        FROM combined_sales AS cs
        JOIN sales_2024 AS s_2024 ON ((cs.subcategory = s_2024.subcategory) AND (cs.category = s_2024.category))
        JOIN sales_2025 AS s_2025 ON ((cs.subcategory = s_2025.subcategory) AND (cs.category = s_2025.category))
        WHERE ((cs.rank <= 25) OR (s_2024.rank <= 25) OR (s_2025.rank <= 25))
        ORDER BY cs.rank ASC;
        '''

    db_handle.connect()
    TOP_CATEGORIES = db_handle.execute_sql(sql, options=db_options.RETURN_RESULTS)
    for row in TOP_CATEGORIES:
        print(row)

    with open('Team_8_Top_25_Categories.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
    
        headers = ['Combined_Rank', 'Combined_Sold', 'Subcategory', 'Category', \
                   '2024_Rank', '2024_Sold', '2025_Rank', '2025_Sold']
    
        writer.writerow(headers)
        writer.writerows(TOP_CATEGORIES)
    db_handle.close()
    
run()

(1, 1153702.0, 'Coffee', 'Hot Beverages', 1, 827860.0, 1, 325842.0)
(2, 1125991.0, 'Chips', 'Snack Foods', 2, 808929.0, 2, 317062.0)
(3, 1082343.0, 'Fresh Vegetables', 'Vegetables', 3, 777464.0, 3, 304879.0)
(4, 746586.0, 'Packaged Snacks', 'Snack Foods', 4, 536029.0, 4, 210557.0)
(5, 637018.0, 'Fresh Fruit', 'Fruit', 5, 457641.0, 5, 179377.0)
(6, 550413.0, 'Pasta', 'Starchy Foods', 6, 395391.0, 6, 155022.0)
(7, 548541.0, 'Cookies', 'Snack Foods', 7, 393828.0, 7, 154713.0)
(8, 469733.0, 'Soup', 'Canned Soup', 8, 337293.0, 8, 132440.0)
(9, 425798.0, 'Cheese', 'Dairy', 9, 305660.0, 9, 120138.0)
(10, 407849.0, 'Canned Vegetables', 'Vegetables', 10, 292989.0, 10, 114860.0)
(11, 399237.0, 'Crackers', 'Snack Foods', 11, 287075.0, 11, 112162.0)
(12, 299788.0, 'Milk', 'Dairy', 12, 225296.0, 16, 74492.0)
(13, 292298.0, 'Flavored Drinks', 'Drinks', 13, 209742.0, 12, 82556.0)
(14, 283468.0, 'Juice', 'Pure Juice Beverages', 14, 203926.0, 14, 79542.0)
(15, 283217.0, 'Dips', 'Snack Foods', 15, 20347

## 5. Make these CSVs more worthy of delivering to a client

Convert to Excel and then auto-fit the columns.
This needs to be done by hand.